In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=9
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_058.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_054', 'Input_C_052', 'Input_C_055', 'Input_C_135',
       'Input_C_059', 'Input_C_060', 'Input_C_056', 'Input_C_053',
       'Input_A4_011'], dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,9)

train_rmse: 0.0018062772782699791
test_rmse: 0.0018484704121483684
test_r2: 0.8802204885524739


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.001788355218182537
[0.0018684381774833917, 0.0017178219841128728, 0.0019609645205860083, 0.0018380530704643044, 0.0018853680084753317, 0.001735840238042914, 0.0015637775092538161, 0.0018964500318850819, 0.0016643424606024285, 0.0017524961809192224]


test_r2_ave: 0.8873953987308474
[0.8551077844283718, 0.8905806651122459, 0.8709836562623341, 0.9017510964422667, 0.8712019868919126, 0.910365635785961, 0.9139949641287496, 0.8855052660467183, 0.8964520152040906, 0.8780109170058238]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.007],0.01)

test_rmse_ave: 0.0017920703220896312
[0.0018500499511447746, 0.0017193800735374853, 0.0019760453490751738, 0.0018906539812752275, 0.0018966281677105343, 0.0017317409901115518, 0.0015563086417196273, 0.0019231104685331066, 0.0016299965300786575, 0.001746789067710173]


test_r2_ave: 0.8870377907260281
[0.8579456631279614, 0.8903820852519976, 0.868991621273978, 0.8960473114344246, 0.86965892795612, 0.9107884853322245, 0.914814551577135, 0.8822634874151826, 0.9006816204765453, 0.8788041534147102]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,9,model)

train_rmse: 0.0018584341791834512
test_rmse: 0.0018916388734179254
test_r2: 0.8746508093208286


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 125, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 2)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.001957723664120399
[0.0018192664567255122, 0.0020426679550993366, 0.0023258002194215246, 0.0019647620449314957, 0.002034236989851194, 0.002044478970358694, 0.0017692717225923286, 0.0019242484325941436, 0.0017134737818254805, 0.0019390300678042809]


test_r2_ave: 0.8652822898976321
[0.8626336962215126, 0.8452846837289415, 0.818511098452525, 0.8877383195800552, 0.8500591423807637, 0.8756572881204217, 0.8899061577240654, 0.8821241095040104, 0.8902483160584287, 0.8506600872055977]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 8,
            min_samples_split = 3,
            max_leaf_nodes = 12,
            min_samples_leaf =4,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0019141087235315817
[0.0018577097224646156, 0.0018288595900760132, 0.0022601966593258542, 0.0019292934406905775, 0.002022019095598604, 0.0020978843597729575, 0.001686142272855897, 0.0018281012695572562, 0.0017198735048260015, 0.001911007320148044]


test_r2_ave: 0.8712026715284589
[0.856766931412636, 0.8759780699202568, 0.8286051709686606, 0.8917549128190362, 0.8518548623985964, 0.8690763423173369, 0.9000086598305587, 0.893609413213082, 0.8894269529794431, 0.8549453994249814]


In [10]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.006], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 125, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 2)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 8,
            min_samples_split = 3,
            max_leaf_nodes = 12,
            min_samples_leaf =4,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0017861589621759894
[0.0018232554045887024, 0.0017351552243524203, 0.0019987596454125833, 0.0018630627867701054, 0.0019011544189298127, 0.0017722800828680154, 0.001554080032509346, 0.0018605550705409794, 0.0016142668825802958, 0.0017390200732076305]


test_r2_ave: 0.8878338470637648
[0.8620306534513071, 0.888361389575336, 0.8659624741166377, 0.8990592327511199, 0.8690360748278563, 0.9065628134268289, 0.9150583453109593, 0.8897984349853385, 0.9025892382679154, 0.879879813924349]


In [12]:

stack_mod = StackingRegressor(regressors=[lasso_mod, lr, elastic_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0018020456866318442
[0.0018957340446473024, 0.0017334666283382201, 0.001995057000329247, 0.0018576415489286234, 0.0019282431042042416, 0.0018098109876488221, 0.0015349866756702811, 0.0019056858830634323, 0.0016238150204153953, 0.001736015973072876]


test_r2_ave: 0.8856615383007233
[0.8508434233612131, 0.8885785698977186, 0.8664586155138088, 0.8996458235419625, 0.865277395496336, 0.902563543688424, 0.9171327024587582, 0.8843873549489075, 0.9014334912688075, 0.8802944628312969]


In [13]:
ave(skew_data[title],y,0.1,stack_mod,0.45,vote_mod,0.45,lasso_mod)

test_rmse_ave: 0.0017834001195378359
[0.0018365087557525126, 0.0017235473440568882, 0.0019772254267270335, 0.0018739359681453945, 0.0018948138424088397, 0.001749593477429051, 0.0015483740674780535, 0.0018793038180884543, 0.0016143720518180368, 0.0017363264434740926]


test_r2_ave: 0.8881503508965188
[0.8600175486098127, 0.8898500782862994, 0.8688351003490502, 0.8978775763695895, 0.869908178704683, 0.9089396445061725, 0.9156809443222792, 0.8875662500087845, 0.9025765452623321, 0.8802516425461846]
